# Install Required Libraries

In [1]:
!pip install gensim

/bin/bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by /bin/bash)

         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.

bash: /opt/anaconda/envs/Python3/lib/libtinfo.so.6: no version information available (required by bash)


# Import Necessary Libraries

In [2]:
#Numerical & Data Manipulation
import numpy as np 
import pandas as pd

#Natural Language Processing
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import string

#Text Processing & Modeling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from gensim.models import Word2Vec
from gensim.models.phrases import Phrases, Phraser
from gensim.models import KeyedVectors



[nltk_data] Downloading package stopwords to
[nltk_data]     /home/faculty/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Load the dataset

In [3]:
#Read the file
data = pd.read_csv("clean_data.csv", encoding="latin-1")
print("Loaded data file {0} with {1} rows and {2} columns".format("clean_data.csv"
                                                                  ,data.shape[0]
                                                                  ,data.shape[1]))

data.head()

Loaded data file clean_data.csv with 89 rows and 7 columns


,Course,Type,topic,Description,Link,word_count,cleaned_desc
0,Artificial Intelligence,Courseware,Getting Started with Enterprise-grade AI,This course covers the foundations of AI for b...,https://skillsbuild.org/college-students/digit...,23,course covers foundations ai business includin...
1,Artificial Intelligence,Courseware,OpenDS4All,OpenDS4All is a project created to accelerate ...,https://github.com/odpi/OpenDS4All/tree/master...,111,opends4all project created accelerate creation...
2,Artificial Intelligence,Courseware,IBM AI Education,Dive into this introductory course and get fam...,https://www.mindspark.org/ibm-ai,571,dive introductory course get familiar basics a...
3,Artificial Intelligence,Courseware,Watson Academy,"IBMÂs portfolio of business-ready tools, appl...",https://www.ibm.com/training/artificial-intell...,175,ibms portfolio business ready tools applicatio...
4,Artificial Intelligence,Courseware,Create a Node-RED starter application,A flow-based programming tool for wiring toget...,https://developer.ibm.com/components/node-red/,47,flow based programming tool wiring together ha...


# Recommendations using technique like Average Word2vec and Genism model

In [4]:
#splitting the processed description into words
corpus = []
for words in data['cleaned_desc']:
    corpus.append(words.split())

In [5]:
from gensim.models import KeyedVectors, Word2Vec
path = 'GoogleNews-vectors-negative300.bin'
google_word2vec = KeyedVectors.load_word2vec_format(path, binary=True, limit=20000)

In [6]:
# Training the corpus with Google Pretrained Model
google_model = Word2Vec(vector_size=300, window=5, min_count=2, workers=-1)
google_model.wv = google_word2vec

# Building vocabulary
google_model.build_vocab(corpus)

# Training the Word2Vec model
google_model.train(corpus, total_examples=google_model.corpus_count, epochs=5)

(0, 0)

In [12]:
def vectors(x):
    # Creating a list for storing the vectors (description into vectors)
    word_embeddings = []

    # Reading each book description
    for line in x['cleaned_desc']:
        avgword2vec = None
        count = 0
        for word in line.split():
            if word in google_model.wv.index_to_key:
                count += 1
                if avgword2vec is None:
                    avgword2vec = google_model.wv[word]
                else:
                    avgword2vec = avgword2vec + google_model.wv[word]
                
        if avgword2vec is not None:
            avgword2vec = avgword2vec / count
            word_embeddings.append(avgword2vec)
    
    return word_embeddings

In [13]:
def recommendations(data):
    # Drop duplicates
    data_unique = data.drop_duplicates(subset='topic')

    # Calling the function vectors
    word_embeddings = vectors(data_unique)

    # Converting the word_embeddings list to array
    array_embeddings = np.array(word_embeddings)

    # Finding cosine similarity for the vectors
    cosine_similarities = cosine_similarity(array_embeddings, array_embeddings)

    # Taking the title and book image link and storing them in a new dataframe called books
    books = data_unique[['topic', 'Link']]

    # Reset the index of books DataFrame
    books.reset_index(drop=True, inplace=True)

    # Reverse mapping of the index
    indices = pd.Series(books.index, index=books['topic']).drop_duplicates()

    # Create a new DataFrame to store the recommendations
    recommendations_df = pd.DataFrame(columns=['Topic', 'Recommended_Course_1', 'Recommended_Course_2', 'Recommended_Course_3', 'Recommended_Course_4', 'Recommended_Course_5'])

    # Iterate through each topic
    for topic in data['topic']:
        idx = indices[topic]
        sim_scores = list(enumerate(cosine_similarities[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        sim_scores = sim_scores[1:6]
        book_indices = [i[0] for i in sim_scores]
        recommend = books.iloc[book_indices]
        
        # Store the recommendations in the recommendations_df
        recommendations_df = recommendations_df.append({'Topic': topic,
                                                        'Recommended_Course_1': recommend.iloc[0]['topic'],
                                                        'Recommended_Course_2': recommend.iloc[1]['topic'],
                                                        'Recommended_Course_3': recommend.iloc[2]['topic'],
                                                        'Recommended_Course_4': recommend.iloc[3]['topic'],
                                                        'Recommended_Course_5': recommend.iloc[4]['topic']},
                                                       ignore_index=True)

    # Save the recommendations to a new CSV file
    recommendations_df.to_csv('recommended_courses_w2v.csv', index=False)


In [14]:
recommendations(data)

/tmp/ipykernel_127563/1346660949.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Topic': topic,
/tmp/ipykernel_127563/1346660949.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Topic': topic,
/tmp/ipykernel_127563/1346660949.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Topic': topic,
/tmp/ipykernel_127563/1346660949.py:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  recommendations_df = recommendations_df.append({'Topic': topic,
/tmp/ipykernel_127563/1346660949.py:36: Futu

In [19]:
#Read the file
recommended_courses = pd.read_csv("recommended_courses_w2v.csv", encoding="latin-1")
print("Loaded data file {0} with {1} rows and {2} columns".format("recommended_courses_w2v.csv"
                                                                  ,recommended_courses.shape[0]
                                                                  ,recommended_courses.shape[1]))

recommended_courses.head()

Loaded data file recommended_courses_w2v.csv with 89 rows and 6 columns


,Topic,Recommended_Course_1,Recommended_Course_2,Recommended_Course_3,Recommended_Course_4,Recommended_Course_5
0,Getting Started with Enterprise-grade AI,IBM AI Education,AI FactSheets 360,OpenDS4All,Watson Academy,AI Fairness 360
1,OpenDS4All,IBM AI Education,Getting Started with Enterprise Data Science,Watson Academy,AI FactSheets 360,Adversarial Robustness Toolbox
2,IBM AI Education,OpenDS4All,Watson Academy,AI FactSheets 360,Getting Started with Enterprise Data Science,Adversarial Robustness Toolbox
3,Watson Academy,IBM AI Education,Getting Started with Enterprise Data Science,Solution tutorials,IBM DB2 Standard,Journey to Cloud: Envisioning Your Solution
4,Create a Node-RED starter application,Watson Academy,IBM AI Education,IBM Planning Analytics - Design and Develop Mo...,Solution tutorials,Getting Started with Enterprise Data Science


In [20]:
# Calculating the percentage of the missing values 
percentage = (recommended_courses.isnull().sum() / recommended_courses.shape[0]).sort_values(ascending=False)
percentage

Topic                   0.0
Recommended_Course_1    0.0
Recommended_Course_2    0.0
Recommended_Course_3    0.0
Recommended_Course_4    0.0
Recommended_Course_5    0.0
dtype: float64

# Load the Test Dataset

In [21]:
#Read the file
test_data = pd.read_csv("test_data.csv", encoding="latin-1")
print("Loaded data file {0} with {1} rows and {2} columns".format("test_data.csv"
                                                                  ,test_data.shape[0]
                                                                  ,test_data.shape[1]))

test_data.head()

Loaded data file test_data.csv with 84 rows and 2 columns


,Topic,label
0,Getting Started with Enterprise Data Science,Getting Started with Enterprise-grade AI
1,Getting Started with Threat Intelligence and H...,Getting Started with Enterprise Data Science
2,IBM Cloud Feature Code,Solution tutorials
3,IBM CloudLabs,Journey to Cloud: Envisioning Your Solution
4,IBM Cognos Analytics - Architecture and Logging,IBM Planning Analytics - Design and Develop Mo...


In [22]:
# Merge the test data with the recommended courses data based on course_name
merged_data = test_data.merge(recommended_courses, on='Topic')
merged_data.to_csv('merged_data_w2v.csv', index=False)

In [23]:
# Read the file
data = pd.read_csv("merged_data_w2v.csv", encoding="latin-1")
data = data.drop_duplicates(keep='first')
print("Loaded data file {0} with {1} rows and {2} columns".format("merged_data_w2v.csv"
                                                                  , data.shape[0]
                                                                  , data.shape[1]))

data.head()

Loaded data file merged_data_w2v.csv with 83 rows and 7 columns


,Topic,label,Recommended_Course_1,Recommended_Course_2,Recommended_Course_3,Recommended_Course_4,Recommended_Course_5
0,Getting Started with Enterprise Data Science,Getting Started with Enterprise-grade AI,Watson Academy,IBM AI Education,SPSS Modeler Premium,OpenDS4All,IBM Planning Analytics - Design and Develop Mo...
2,Getting Started with Threat Intelligence and H...,Getting Started with Enterprise Data Science,QRadar SIEM,Elyra,IBM DB2 Standard,Watson Academy,SPSS Modeler Premium
4,IBM Cloud Feature Code,Solution tutorials,Watson Academy,IBM DB2 Standard,IBM Cloud Associate Solution Advisor,Journey to Cloud: Envisioning Your Solution,SPSS Modeler Premium
5,IBM CloudLabs,Journey to Cloud: Envisioning Your Solution,Solution tutorials,Watson Academy,Journey to Cloud: Envisioning Your Solution,IBM DB2 Standard,Getting Started with Cloud for the Enterprise
6,IBM Cognos Analytics - Architecture and Logging,IBM Planning Analytics - Design and Develop Mo...,IBM Cognos Analytics - Author Reports with Mul...,Overview of IBM Cognos Analytics,z/OS Introduction,QRadar SIEM,Journey to Cloud: Envisioning Your Solution


# Calculate the Accuracy

In [25]:
# Define function to calculate the top-k accuracy
def calculate_accuracy(dataset, k):
    accuracies = []
    topics_with_accuracy_1 = []
    
    for i, row in dataset.iterrows():
        recommended_courses = row[['Recommended_Course_1', 'Recommended_Course_2',
                                   'Recommended_Course_3', 'Recommended_Course_4',
                                   'Recommended_Course_5']].tolist()
        recommended_courses = [course for course in recommended_courses if course != 'N/A']
        label = row['label']
        
        if label in recommended_courses[:k]:
            accuracies.append(1)
            topics_with_accuracy_1.append(row['Topic'])
        else:
            accuracies.append(0)
    
    average_accuracy = np.mean(accuracies) * 100
    average_accuracy = round(average_accuracy, 2)
    return average_accuracy, topics_with_accuracy_1


In [28]:
average_accuracy, topics_with_accuracy_1 = calculate_accuracy(data, 1)
print(f"Average Accuracy for k = 1: {average_accuracy}%")

Average Accuracy for k = 1: 65.06%


In [27]:
# Print topics having accuracy = 1 for k = 1
if topics_with_accuracy_1:
    print("Topics with Accuracy = 1:")
    for topic in topics_with_accuracy_1:
        print(topic)
else:
    print("No topics with Accuracy = 1 found.")

Topics with Accuracy = 1:
Adversarial Robustness Toolbox
AI Explainability 360
AI FactSheets 360
AI Fairness 360
Application Modernization with IBM z/OS
Architecting Applications with IBM Z
Causal Inference 360
Cognos Analytics
Constraint Programming with ILOG CP Optimizer
Create a Node-RED starter application
Developing Secure Software
DevOps Transformations for IBM zSystems and CICD pipelines with DBB Git
Elyra
Explore the IBM Software Catalog
Fundamentals of IBM Process Mining
Getting Started with Cloud for the Enterprise
Getting Started with Enterprise-grade AI
IBM AI Education
IBM Analog Hardware Acceleration Kit
IBM Cloud Associate Solution Advisor
IBM Digital App Builder
IBM Engineering Workflow Management
IBM Maximo Visual Inspection
IBM Quantum Lab & Composer
IBM Robotic Process Automation - Basic II
IBM Z System Administrator
Introduction to Quantum Computing and Quantum Hardware
Journey to Cloud: Envisioning Your Solution
OpenDS4All
OpenLiberty
Overview of IBM Cognos Analyti

In [29]:
average_accuracy, topics_with_accuracy_3 = calculate_accuracy(data, 3)
print(f"Average Accuracy for k = 3: {average_accuracy}%")

Average Accuracy for k = 3: 66.27%


In [30]:
average_accuracy, topics_with_accuracy_5 = calculate_accuracy(data, 5)
print(f"Average Accuracy for k = 5: {average_accuracy}%")

Average Accuracy for k = 5: 67.47%
